**discountのCATE**
median_playtime_forever

炎上の影響を高評価率によって引きたい

CCUではなく、他の妥当な比説明変数を使いたいが、量（Review）と質（プレイ時間中央値）の両方の関係も見たい

それらを踏まえた上で割引のCATEを測りたい

In [ ]:
import pandas as pd
import numpy as np
from econml.dml import LinearDML
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt

# 1. データの準備（Sand_03 を想定）
# T: 処置（割引率 0-100）
# X: 条件（CATEを見たい変数：メタスコアなど）
# W: 共変量（掃除したい変数：ポジティブ率、価格、タグ、ジャンルなど）
# Y_vol: 量（レビュー数） / Y_qual: 質（中央値プレイ時間）

T = df['discount'].values
X = df[['metacritic_score']].values  # メタスコアによって効果が変わるか見たい
W = df[['pct_pos_total', 'price', 'dlc_count', 'is_early_access']].values # 炎上率等で掃除

# 目的変数は対数変換（0を避けるため +1）
Y_vol = np.log(df['num_reviews_total'] + 1).values
Y_qual = np.log(df['median_playtime_forever'] + 1).values

# 2. モデルの定義（Double Machine Learning）
# 機械学習（Random Forest）で T と Y のノイズを掃除する
def estimate_cate(Y, T, X, W):
    est = LinearDML(
        model_y=RandomForestRegressor(n_estimators=100, max_depth=5, min_samples_leaf=10),
        model_t=RandomForestRegressor(n_estimators=100, max_depth=5, min_samples_leaf=10),
        discrete_treatment=False
    )
    est.fit(Y, T, X=X, W=W)
    return est

# 3. 「量」と「質」のそれぞれで CATE を推定
print("Estimating Volume Effect (Reviews)...")
est_vol = estimate_cate(Y_vol, T, X, W)

print("Estimating Quality Effect (Playtime)...")
est_qual = estimate_cate(Y_qual, T, X, W)

# 4. 可視化：メタスコアが上がると、割引の「量」と「質」への効果はどう変わるか
test_X = np.linspace(X.min(), X.max(), 100).reshape(-1, 1)
cate_vol = est_vol.effect(test_X)
cate_qual = est_qual.effect(test_X)

plt.figure(figsize=(10, 6))
plt.plot(test_X, cate_vol, label='CATE: Volume (Reviews)', color='blue')
plt.plot(test_X, cate_qual, label='CATE: Quality (Playtime)', color='green')
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('Metacritic Score')
plt.ylabel('Marginal Treatment Effect of Discount')
plt.title('Quantity vs. Quality Trade-off in Discounting')
plt.legend()
plt.show()

でもこれ同時方程式じゃ。IVーDML使った方がいいんじゃないか。

いや売れてるゲームほど割引するって仮説は繊細すぎるか、一旦この2変数とdiscountの相関見るか。だって売れてないゲームも割引するだろうから打ち消されそうだ

でもそもそもマジで一応市場に出ただけみたいなクソゲーもdiscountしないだろうしやっぱりreviewをheckmanで切りたくなってしまうな。難しい

Low-end (クソゲー): そもそも割引すら設定されず放置される。

Mid-range (普通): 売れないから割引する（負の相関）。

High-end (人気作): セールイベントで戦略的に割引し、さらに売れる（正の相関）。

これらがデータセット内で混ざると、全体の係数は「0」に近づき、「割引は意味がない」という誤った結論を導きます。

4. 暫定的な解決策：Heckman-DML 的なアプローチ

最新の論文でも議論される内容ですが、現実的な解として以下を提案します。

Step 1: フィルタリング（Heckman の精神）
「レビューが 10 以上」などの最低限の足切りを行い、さらに「過去に一度でも割引をしたことがあるか」というプロビットを回して IMR（逆ミルズ比） を出す。

Step 2: DML への投入
その IMR をコントロール変数として DML にぶち込みます。これで「割引戦略をとるようなゲーム層」における、純粋な割引効果を推定できます。

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import norm
from econml.dml import LinearDML
from sklearn.ensemble import RandomForestRegressor

# 1. Selection Stage: 逆ミルズ比 (IMR) の算出
# 全データを使用して「一定以上のレビューがあるか」という生存フラグを作成
df['is_observed'] = (df['num_reviews_total'] > 10).astype(int)

# 選択方程式の変数 Z (排外制約: 生存には効くが、売上そのものには直接関わらないもの)
# 例: 発売からの日数、言語数、特定のカテゴリフラグなど
Z_cols = ['language_count', 'is_early_access', 'windows', 'mac', 'linux']
Z = sm.add_constant(df[Z_cols])

# プロビット回帰の実行
probit_model = sm.Probit(df['is_observed'], Z).fit()

# 各サンプルに対して逆ミルズ比 (IMR) を計算
# phi(z)/Phi(z)
z_score = probit_model.predict(Z)
# predictは確率を出すので、逆関数で標準正規分布のzスコアに戻す必要がある
z_val = norm.ppf(np.clip(z_score, 0.001, 0.999))
df['imr'] = norm.pdf(z_val) / norm.cdf(z_val)

# 2. DML Stage: 選択バイアスを補正した CATE 推定
# 分析対象を「観測されたデータ」に絞る
df_sub = df[df['is_observed'] == 1].copy().dropna(subset=['metacritic_score', 'discount'])

# Y: 目的変数 (量: reviews / 質: playtime)
Y = np.log(df_sub['num_reviews_total'] + 1).values

# T: 処置変数 (割引率)
T = df_sub['discount'].values

# X: 条件変数 (CATEを見たい軸)
X = df_sub[['metacritic_score']].values

# W: 共変量 (IMRをここに追加するのがキモ)
# 他にコントロールしたい変数（炎上率など）も入れる
W = df_sub[['pct_pos_total', 'price', 'imr']].values

# LinearDML の定義
est = LinearDML(
    model_y=RandomForestRegressor(n_estimators=100, max_depth=5),
    model_t=RandomForestRegressor(n_estimators=100, max_depth=5),
    discrete_treatment=False
)

# 学習
print("Fitting Heckman-DML model...")
est.fit(Y, T, X=X, W=W)

# 3. 結果の解釈
# メタスコアごとの割引の限界効果を表示
test_X = np.linspace(X.min(), X.max(), 100).reshape(-1, 1)
treatment_effects = est.effect(test_X)

import matplotlib.pyplot as plt
plt.plot(test_X, treatment_effects)
plt.axhline(0, color='r', linestyle='--')
plt.title("CATE of Discount (Heckman-DML Corrected)")
plt.xlabel("Metacritic Score")
plt.ylabel("Treatment Effect on Log Reviews")
plt.show()